In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from time import time
from random import randint
from math import ceil

import torch

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [5]:
tokenizer = AutoTokenizer.from_pretrained("Vamsi/T5_Paraphrase_Paws")  
model = AutoModelForSeq2SeqLM.from_pretrained("Vamsi/T5_Paraphrase_Paws").to(device)

In [6]:
text = 'A.C. Monza is a professional football club that is based in Monza, Lombardy, Italy. The team plays in the Serie A, the first tier of Italian football, following promotion in the 2021–22 Serie B season. The club was founded in 1912 (first lineup pictured), with its first recorded win on 20 September 1912. On multiple occasions in the 1970s, the club came close to promotion to the Serie A, but were twice declared bankrupt, in 2004 and 2015. Following Silvio Berlusconi\'s 2018 takeover of the club.'

In [7]:
def paraphrase(sentence: str = '', level: str = 'normal'):
    max_length = 500 if len(sentence) > 500 else len(sentence)

    encoding = tokenizer.encode_plus(sentence,padding='longest', return_tensors="pt")
    input_ids, attention_masks = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)

    early_stopping = False

    if level == 'normal' or level == 'creative':
        early_stopping = True

    outputs = model.generate(
        input_ids=input_ids,
        attention_mask=attention_masks,
        num_return_sequences=4,
        max_length=max_length,
        do_sample=True,
        top_k=int(max_length / 2),
        top_p=0.95,
        early_stopping=early_stopping,
    )

    shortest = ''
    shortest_counter = max_length
    longest = ''
    longest_counter = 0

    for output in outputs:
        line = tokenizer.decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=True)

        if line[-1] == '.':
            if len(line) <= shortest_counter:
                shortest = line
                shortest_counter = len(line)

            if len(line) >= longest_counter:
                longest = line
                longest_counter = len(line)

    if level in [ 'normal', 'creative', 'shortest' ]:
        return ( shortest, len(shortest) )

    return ( longest, len(longest) )

In [9]:
print("-"*100)
print("Input: ", text)
print("-"*100)

print("")

print("-"*100)
print("Level: Normal")
print("-"*100)

start = time()

results = paraphrase(sentence=text, level='normal')

for line in results:
    print(line)

print("-"*100)
print("Elapsed: ", time() - start)
print("")

print("-"*100)
print("Level: Creative")
print("-"*100)

start = time()

results = paraphrase(sentence=text, level='creative')

for line in results:
    print(line)

print("-"*100)
print("Elapsed: ", time() - start)
print("")

print("-"*100)
print("Level: Fluent")
print("-"*100)

start = time()

results = paraphrase(sentence=text, level='fluent')

for line in results:
    print(line)

print("-"*100)
print("Elapsed: ", time() - start)
print("")

print("-"*100)
print("Level: Longest")
print("-"*100)

start = time()

results = paraphrase(sentence=text, level='longest')

for line in results:
    print(line)

print("-"*100)
print("Elapsed: ", time() - start)
print("")

print("-"*100)
print("Level: Shortest")
print("-"*100)

start = time()

results = paraphrase(sentence=text, level='shortest')

for line in results:
    print(line)

print("-"*100)
print("Elapsed: ", time() - start)
print("")

----------------------------------------------------------------------------------------------------
Input:  A.C. Monza is a professional football club that is based in Monza, Lombardy, Italy. The team plays in the Serie A, the first tier of Italian football, following promotion in the 2021–22 Serie B season. The club was founded in 1912 (first lineup pictured), with its first recorded win on 20 September 1912. On multiple occasions in the 1970s, the club came close to promotion to the Serie A, but were twice declared bankrupt, in 2004 and 2015. Following Silvio Berlusconi's 2018 takeover of the club.
----------------------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------
Level: Normal
----------------------------------------------------------------------------------------------------
A.C. Monza is a professional football club based in Monza, Lombardy, Ital